# Position MC Report

This notebook calculates Marginal Contribution (MC) to total portfolio risk broken down by position/strategy from `pos_summary.csv`, with separate sections for single-product analysis and cross-product aggregation.

In [100]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## 1. Configuration & Data Loading

In [101]:
# ============================================================================
# CONFIGURATION - Reuse from q_risk_report.ipynb
# ============================================================================

# Bucket definitions
front = ["A01", "A02", "A03", "A04"]
mid   = ["A05", "A06", "A07", "A08"]
back  = ["A09", "A10", "A11", "A12", "A13", "A14", "A15"]

# EWMA decay parameters (lambda)
lambda_front = 0.97
lambda_mid   = 0.98
lambda_back  = 0.99

# EWMA initialization
ewma_init_obs = 60  # Use first N observations for sample covariance initialization

# Data file paths
data_file = "data_.csv"
delta_summary_file = "delta_summary.csv"
delta_positions_file = "delta_positions.csv"
pos_summary_file = "pos_summary.csv"
product_mapping_file = "product_mapping.csv"
holidays_file = "holidays.csv"

# Load all data files
print("Loading data files...")
pos_summary_df = pd.read_csv(pos_summary_file)
delta_positions_df = pd.read_csv(delta_positions_file)
delta_summary_df = pd.read_csv(delta_summary_file)
df_raw = pd.read_csv(data_file)
product_mapping_df = pd.read_csv(product_mapping_file)

# Load holidays and create set for fast lookup
holidays_df = pd.read_csv(holidays_file, header=None, names=['date'])
holidays_df = holidays_df.dropna()  # Remove empty rows
holidays_df['date'] = pd.to_datetime(holidays_df['date'], format='%m/%d/%Y', errors='coerce')
holidays_df = holidays_df.dropna()  # Remove any rows that couldn't be parsed
holiday_dates = set(holidays_df['date'].dt.date)  # Use .date() for date-only comparison

# Parse date column for price data
df_raw['date'] = pd.to_datetime(df_raw['date'])
df_raw = df_raw.set_index('date').sort_index()

# Build contract-to-node mapping (same as q_risk_report.ipynb)
# Map tenors chronologically to A01-A15 (first 15 tenors in delta_summary.csv)
unique_tenors = delta_summary_df['Tenor'].unique()
contract_to_node = {}
for idx, tenor in enumerate(unique_tenors[:15]):  # Only map first 15 to A01-A15
    node_code = f"A{idx+1:02d}"
    contract_to_node[tenor] = node_code

# All nodes
all_nodes = [f"A{i:02d}" for i in range(1, 16)]
n_total = len(all_nodes)

# Build product mapping dictionary from CSV
product_to_mapped = dict(zip(product_mapping_df['Product'], product_mapping_df['Mapping']))

# Build mapping from Mapped_Product to lowercase data column prefix
# This maps mapped products (HTT, HOUBR, etc.) to their lowercase data column names (htt, houbr, etc.)
mapped_to_data_column = {
    'HTT': 'htt',
    'HOUBR': 'houbr',
    'CLBR': 'clbr',
    'WDF': 'wdf',
    'LH': 'lh'  # Longhorn
}

print(f"Loaded {len(pos_summary_df)} positions from pos_summary.csv")
print(f"Loaded {len(delta_positions_df)} expanded positions from delta_positions.csv")
print(f"Loaded {len(delta_summary_df)} tenors from delta_summary.csv")
print(f"Loaded {len(product_mapping_df)} product mappings from product_mapping.csv")
print(f"Loaded {len(holiday_dates)} holiday dates from holidays.csv")
print(f"Contract-to-node mappings: {len(contract_to_node)} contracts")
print(f"\nProduct mapping (Product -> Mapped_Product):")
for orig, mapped in product_to_mapped.items():
    print(f"  {orig} -> {mapped}")
print(f"\nUnique strategies: {sorted(pos_summary_df['Strategy'].unique())}")
print(f"Unique products: {sorted(pos_summary_df['Product'].unique())}")

Loading data files...
Loaded 20 positions from pos_summary.csv
Loaded 100 expanded positions from delta_positions.csv
Loaded 34 tenors from delta_summary.csv
Loaded 9 product mappings from product_mapping.csv
Loaded 111 holiday dates from holidays.csv
Contract-to-node mappings: 15 contracts

Product mapping (Product -> Mapped_Product):
  HTT -> HTT
  HOUBR_Cross -> HOUBR
  HOUBR -> HOUBR
  CLBR -> CLBR
  HTT Rolls -> HTT
  HOUBR Boxes -> HOUBR
  CLBR Boxes -> CLBR
  HTTMID -> LH
  WDF -> WDF

Unique strategies: ['Freight', 'HOUBR_Back', 'HOUBR_Front', 'HOUBR_rolls', 'HTT_Back', 'HTT_Front', 'HTT_Mid', 'HTT_Rolls', 'Longhorn']
Unique products: ['CLBR', 'CLBR Boxes', 'HOUBR', 'HOUBR Boxes', 'HOUBR_Cross', 'HTT', 'HTT Rolls', 'HTTMID', 'WDF']


## 2. Reuse Functions from q_risk_report.ipynb

In [102]:
def compute_ewma_covariance(returns_df, nodes, product, lambda_val, init_obs=60):
    """
    Compute EWMA covariance matrix for given nodes.
    
    Parameters:
    -----------
    returns_df : DataFrame
        DataFrame with returns (daily changes)
    nodes : list
        List of node names (e.g., ['A01', 'A02', ...])
    product : str
        Product name (e.g., 'htt', 'houbr', etc.)
    lambda_val : float
        EWMA decay parameter
    init_obs : int
        Number of observations to use for initial covariance
    
    Returns:
    --------
    cov_matrix : ndarray
        Final EWMA covariance matrix
    """
    # Extract relevant columns
    cols = [f'{product}_{node}' for node in nodes]
    returns_subset = returns_df[cols].values
    
    n_obs, n_nodes = returns_subset.shape
    
    if n_obs < init_obs:
        raise ValueError(f"Need at least {init_obs} observations, got {n_obs}")
    
    # Initialize with sample covariance of first N observations
    init_returns = returns_subset[:init_obs]
    # Remove any rows with NaN
    init_returns = init_returns[~np.isnan(init_returns).any(axis=1)]
    
    if len(init_returns) < 10:
        # Fallback: use identity matrix scaled by variance
        cov_current = np.eye(n_nodes) * np.var(returns_subset, axis=0).mean()
    else:
        cov_current = np.cov(init_returns.T)
    
    # EWMA recursion: Σ_t = λ * Σ_{t-1} + (1-λ) * r_t * r_t'
    for t in range(init_obs, n_obs):
        r_t = returns_subset[t:t+1, :]  # Shape: (1, n_nodes)
        
        # Skip if any NaN
        if np.isnan(r_t).any():
            continue
        
        # EWMA update
        outer_product = np.outer(r_t, r_t)
        cov_current = lambda_val * cov_current + (1 - lambda_val) * outer_product[0]
    
    return cov_current

def compute_mc_to_total(w_position, w_total, Sigma_total):
    """
    Compute marginal contribution of a position to total portfolio.
    MC = 1000 * (w_position' Σ_total w_total) / sqrt(w_total' Σ_total w_total)
    
    Parameters:
    -----------
    w_position : ndarray
        Position vector (15 nodes)
    w_total : ndarray
        Total portfolio vector (15 nodes)
    Sigma_total : ndarray
        Total covariance matrix (15x15)
    
    Returns:
    --------
    mc : float
        Marginal contribution to total portfolio risk
    """
    # Compute numerator: w_position' Σ_total w_total
    numerator = w_position.T @ Sigma_total @ w_total
    
    # Compute denominator: sqrt(w_total' Σ_total w_total)
    total_var = w_total.T @ Sigma_total @ w_total
    if total_var <= 0:
        return 0.0
    denominator = np.sqrt(total_var)
    
    return 1000 * numerator / denominator

print("Functions loaded: compute_ewma_covariance, compute_mc_to_total")

Functions loaded: compute_ewma_covariance, compute_mc_to_total


## 3. Strategy Position Mapping

In [103]:
def build_strategy_position_vector(strategy_name, product, delta_positions_df, contract_to_node, all_nodes):
    """
    Build position vector for a strategy within a product.
    
    Parameters:
    -----------
    strategy_name : str
        Strategy name (e.g., 'HTT_Front')
    product : str
        Mapped product name (e.g., 'HTT', 'HOUBR', etc.)
    delta_positions_df : DataFrame
        Expanded positions with Strategy and Mapped_Product columns
    contract_to_node : dict
        Mapping from tenor to node code
    all_nodes : list
        List of all node codes (A01-A15)
    
    Returns:
    --------
    w_strategy : ndarray
        Position vector (15 nodes) for this strategy
    """
    # Filter delta_positions for this strategy and product
    strategy_positions = delta_positions_df[
        (delta_positions_df['Strategy'] == strategy_name) & 
        (delta_positions_df['Mapped_Product'] == product)
    ].copy()
    
    # Initialize position vector
    w_strategy = np.zeros(len(all_nodes))
    
    # Map each tenor to node and accumulate quantities
    for _, row in strategy_positions.iterrows():
        tenor = row['Tenor']
        qty = row['Qty']
        
        if tenor in contract_to_node:
            node = contract_to_node[tenor]
            node_idx = all_nodes.index(node)
            w_strategy[node_idx] += qty
    
    return w_strategy

def determine_bucket(w_position, front, mid, back, all_nodes):
    """
    Determine which bucket(s) a position belongs to based on non-zero nodes.
    
    Returns:
    --------
    bucket : str
        'Front', 'Mid', 'Back', 'Mixed', or 'None'
    """
    front_positions = w_position[[all_nodes.index(n) for n in front]]
    mid_positions = w_position[[all_nodes.index(n) for n in mid]]
    back_positions = w_position[[all_nodes.index(n) for n in back]]
    
    has_front = np.any(np.abs(front_positions) > 1e-10)
    has_mid = np.any(np.abs(mid_positions) > 1e-10)
    has_back = np.any(np.abs(back_positions) > 1e-10)
    
    buckets = []
    if has_front:
        buckets.append('Front')
    if has_mid:
        buckets.append('Mid')
    if has_back:
        buckets.append('Back')
    
    if len(buckets) == 0:
        return 'None'
    elif len(buckets) == 1:
        return buckets[0]
    else:
        return 'Mixed'

print("Strategy mapping functions loaded")

Strategy mapping functions loaded


## 4. Single Product MC Calculation

In [104]:
# Get all unique mapped products from delta_positions
all_products = sorted(delta_positions_df['Mapped_Product'].unique())

print(f"Products to process: {all_products}")
print(f"Mapped Product -> Data Column mapping: {mapped_to_data_column}")

# Store results
position_mc_results = []
diagnostic_data = []  # Store diagnostic information

# Process each product
for mapped_product in all_products:
    product_lower = mapped_to_data_column.get(mapped_product, mapped_product.lower())
    
    print(f"\n{'='*80}")
    print(f"Processing product: {mapped_product} (data column: {product_lower})")
    print(f"{'='*80}")
    
    # Check if product data exists
    product_cols = [col for col in df_raw.columns if col.startswith(f'{product_lower}_A') and '/' not in col]
    if not product_cols:
        print(f"  WARNING: No data columns found for {product_lower}, skipping...")
        continue
    
    # Extract price data for this product
    product_cols = sorted(product_cols, key=lambda x: int(x.split('_A')[1]))
    df_prices = df_raw[product_cols].copy()
    
    # Compute daily changes (returns)
    df_returns = df_prices.diff().dropna()
    df_returns = df_returns.dropna()
    
    # Filter out holiday dates
    initial_count = len(df_returns)
    # Convert index dates to a list and check membership
    is_holiday = [date.date() in holiday_dates for date in df_returns.index]
    df_returns = df_returns[~pd.Series(is_holiday, index=df_returns.index)]
    removed_count = initial_count - len(df_returns)
    
    if len(df_returns) < ewma_init_obs:
        print(f"  WARNING: Insufficient data ({len(df_returns)} < {ewma_init_obs}), skipping...")
        continue
    
    print(f"  Returns data shape: {df_returns.shape}")
    print(f"  Removed {removed_count} holiday dates from returns data (out of {initial_count} total)")
    
    # Compute EWMA covariance for each bucket
    print("  Computing EWMA covariances...")
    Sigma_front = compute_ewma_covariance(df_returns, front, product_lower, lambda_front, ewma_init_obs)
    Sigma_mid = compute_ewma_covariance(df_returns, mid, product_lower, lambda_mid, ewma_init_obs)
    Sigma_back = compute_ewma_covariance(df_returns, back, product_lower, lambda_back, ewma_init_obs)
    
    # Build total covariance matrix (block-diagonal)
    n_front = len(front)
    n_mid = len(mid)
    n_back = len(back)
    
    Sigma_total = np.zeros((n_total, n_total))
    Sigma_total[:n_front, :n_front] = Sigma_front
    Sigma_total[n_front:n_front+n_mid, n_front:n_front+n_mid] = Sigma_mid
    Sigma_total[n_front+n_mid:, n_front+n_mid:] = Sigma_back
    
    print(f"  Total covariance shape: {Sigma_total.shape}")
    
    # Build total portfolio position vector for this product from delta_summary
    w_total = np.zeros(n_total)
    if mapped_product in delta_summary_df.columns:
        for _, row in delta_summary_df.iterrows():
            tenor = row['Tenor']
            position = row[mapped_product]
            if abs(position) > 1e-10 and tenor in contract_to_node:
                node = contract_to_node[tenor]
                node_idx = all_nodes.index(node)
                w_total[node_idx] = float(position)
    
    total_qty = np.sum(np.abs(w_total))
    print(f"  Total portfolio quantity: {total_qty:.2f}")
    
    # Display position vector
    print(f"  Position vector (nodes A01-A15):")
    for i, node in enumerate(all_nodes):
        if abs(w_total[i]) > 1e-10:
            print(f"    {node}: {w_total[i]:.2f}")
    
    # Get all strategies for this product
    product_strategies = delta_positions_df[
        delta_positions_df['Mapped_Product'] == mapped_product
    ]['Strategy'].unique()
    
    print(f"  Found {len(product_strategies)} strategies: {sorted(product_strategies)}")
    
    # Calculate MC for each strategy
    for strategy_name in sorted(product_strategies):
        # Build strategy position vector
        w_strategy = build_strategy_position_vector(
            strategy_name, mapped_product, delta_positions_df, 
            contract_to_node, all_nodes
        )
        
        strategy_qty = np.sum(np.abs(w_strategy))
        if strategy_qty < 1e-10:
            continue  # Skip empty strategies
        
        # Calculate MC
        mc = compute_mc_to_total(w_strategy, w_total, Sigma_total)
        
        # Determine bucket
        bucket = determine_bucket(w_strategy, front, mid, back, all_nodes)
        
        # Count unique tenors for this strategy
        strategy_tenors = delta_positions_df[
            (delta_positions_df['Strategy'] == strategy_name) & 
            (delta_positions_df['Mapped_Product'] == mapped_product)
        ]['Tenor'].nunique()
        
        # Store result
        position_mc_results.append({
            'Strategy': strategy_name,
            'Product': mapped_product,
            'MC_to_total': mc,
            'Qty_total': strategy_qty,
            'Tenor_count': strategy_tenors,
            'Bucket': bucket
        })
        
        print(f"    {strategy_name}: MC={mc:.2f}, Qty={strategy_qty:.2f}, Bucket={bucket}")

print(f"\n{'='*80}")
print(f"Completed processing all products")
print(f"Total strategy/product combinations: {len(position_mc_results)}")
print(f"{'='*80}")

Products to process: ['CLBR', 'HOUBR', 'HTT', 'LH', 'WDF']
Mapped Product -> Data Column mapping: {'HTT': 'htt', 'HOUBR': 'houbr', 'CLBR': 'clbr', 'WDF': 'wdf', 'LH': 'lh'}

Processing product: CLBR (data column: clbr)
  Returns data shape: (968, 15)
  Removed 10 holiday dates from returns data (out of 978 total)
  Computing EWMA covariances...
  Total covariance shape: (15, 15)
  Total portfolio quantity: 2368.00
  Position vector (nodes A01-A15):
    A02: -268.00
    A05: 300.00
    A07: 850.00
    A08: -850.00
    A10: 100.00
  Found 2 strategies: ['HOUBR_Back', 'HOUBR_rolls']
    HOUBR_Back: MC=-2186.87, Qty=32.00, Bucket=Front
    HOUBR_rolls: MC=35086.98, Qty=2400.00, Bucket=Mixed

Processing product: HOUBR (data column: houbr)
  Returns data shape: (964, 15)
  Removed 8 holiday dates from returns data (out of 972 total)
  Computing EWMA covariances...
  Total covariance shape: (15, 15)
  Total portfolio quantity: 3925.00
  Position vector (nodes A01-A15):
    A01: 300.00
    A02

## 5. Results DataFrame

In [105]:
# Check if diagnostic_data exists (from running cell 9)
if 'diagnostic_data' not in globals() or len(diagnostic_data) == 0:
    print("WARNING: diagnostic_data not found. Please run cell 9 (Single Product MC Calculation) first.")
    print("Skipping diagnostics...")
    diagnostic_df = pd.DataFrame()
else:
    # Create diagnostic DataFrame
    diagnostic_df = pd.DataFrame(diagnostic_data)
    
    print("="*80)
    print("MC CALCULATION DIAGNOSTICS")
    print("="*80)
    
    # Display diagnostic breakdown for each strategy
    print("\nDetailed MC Calculation Components:")
    print("-"*80)
    for _, row in diagnostic_df.iterrows():
        print(f"\n{row['Strategy']} ({row['Product']}):")
        print(f"  Strategy Q (standalone): {row['Strategy_Q_standalone']:.2f}")
        print(f"  Total Portfolio Q: {row['Total_Q_portfolio']:.2f}")
        print(f"  MC Numerator (w_strategy' Σ w_total): {row['MC_numerator']:.2f}")
        print(f"  MC Denominator (sqrt(w_total' Σ w_total)): {row['MC_denominator']:.2f}")
        print(f"  MC Ratio (numerator/denominator): {row['MC_ratio']:.2f}")
        print(f"  Final MC (1000 * ratio): {row['MC_to_total']:.2f}")
        print(f"  Strategy Variance: {row['Strategy_var']:.6f}")
        print(f"  Total Portfolio Variance: {row['Total_var']:.6f}")
    
    # Compare high MC strategies
    print("\n" + "="*80)
    print("COMPARISON: High MC Strategies vs HTT")
    print("="*80)
    high_mc_strategies = diagnostic_df.nlargest(3, 'MC_to_total', keep='all')
    htt_strategies = diagnostic_df[diagnostic_df['Product'] == 'HTT'].nlargest(3, 'MC_to_total', keep='all')
    
    print("\nTop 3 Highest MC Strategies:")
    print(high_mc_strategies[['Strategy', 'Product', 'MC_to_total', 'Strategy_Q_standalone', 
                              'Total_Q_portfolio', 'MC_ratio', 'Qty_total']].to_string(index=False))
    
    print("\nTop 3 HTT Strategies (for comparison):")
    print(htt_strategies[['Strategy', 'Product', 'MC_to_total', 'Strategy_Q_standalone', 
                          'Total_Q_portfolio', 'MC_ratio', 'Qty_total']].to_string(index=False))

Skipping diagnostics...


## 8. Multi-Product Covariance Matrix Analysis

This section demonstrates MC calculation using a multi-product covariance matrix that accounts for cross-product correlations, compared to the independence assumption used in Section 7.

In [106]:
# Create position MC DataFrame
position_mc_df = pd.DataFrame(position_mc_results)

# Add MC sign indicator
position_mc_df['MC_signed'] = position_mc_df['MC_to_total'].apply(
    lambda x: 'POS' if x > 0 else 'NEG' if x < 0 else 'ZERO'
)

# Sort by absolute MC (descending)
position_mc_df['abs_MC'] = position_mc_df['MC_to_total'].abs()
position_mc_df = position_mc_df.sort_values('abs_MC', ascending=False).reset_index(drop=True)
position_mc_df = position_mc_df.drop('abs_MC', axis=1)

print("Position MC DataFrame:")
print(position_mc_df.to_string(index=False))
print(f"\nTotal rows: {len(position_mc_df)}")

# Build and display position vectors summary for all products
print("\n" + "="*80)
print("POSITION VECTORS BY PRODUCT (w_total for each product)")
print("="*80)

# Rebuild position vectors for display
product_position_vectors = {}
for mapped_product in sorted(delta_positions_df['Mapped_Product'].unique()):
    w_total = np.zeros(n_total)
    if mapped_product in delta_summary_df.columns:
        for _, row in delta_summary_df.iterrows():
            tenor = row['Tenor']
            position = row[mapped_product]
            if abs(position) > 1e-10 and tenor in contract_to_node:
                node = contract_to_node[tenor]
                node_idx = all_nodes.index(node)
                w_total[node_idx] = float(position)
    
    # Only show if there are positions
    total_qty = np.sum(np.abs(w_total))
    if total_qty > 1e-10:
        product_position_vectors[mapped_product] = w_total.copy()
        print(f"\n{mapped_product} Position Vector (Total Qty: {total_qty:.2f}):")
        print(f"  Node positions:")
        for i, node in enumerate(all_nodes):
            if abs(w_total[i]) > 1e-10:
                print(f"    {node}: {w_total[i]:.2f}")
        
        # Also show as a compact vector
        non_zero_nodes = [f"{node}={w_total[i]:.2f}" for i, node in enumerate(all_nodes) if abs(w_total[i]) > 1e-10]
        print(f"  Compact: {', '.join(non_zero_nodes)}")

# Create a DataFrame showing position vectors as a table
if product_position_vectors:
    position_vectors_df = pd.DataFrame(product_position_vectors, index=all_nodes).T
    # Only show non-zero columns
    non_zero_cols = [col for col in position_vectors_df.columns if position_vectors_df[col].abs().sum() > 1e-10]
    position_vectors_df = position_vectors_df[non_zero_cols]
    
    print("\n" + "="*80)
    print("POSITION VECTORS TABLE (Products × Nodes)")
    print("="*80)
    print(position_vectors_df.to_string())

Position MC DataFrame:
   Strategy Product   MC_to_total  Qty_total  Tenor_count Bucket MC_signed
  HTT_Rolls     HTT  1.765134e+05  8900.0000           30  Mixed       POS
HOUBR_rolls    CLBR  3.508698e+04  2400.0000            6  Mixed       POS
 HOUBR_Back   HOUBR  2.155257e+04   725.0000           15  Mixed       POS
    HTT_Mid     HTT  1.641687e+04   574.0000            1  Front       POS
   HTT_Back     HTT  5.510005e+03   975.0000           18  Mixed       POS
  HTT_Front     HTT -2.860082e+03   100.0000            1  Front       NEG
 HOUBR_Back    CLBR -2.186870e+03    31.9996            1  Front       NEG
HOUBR_rolls   HOUBR  3.012441e-07  2600.0000            2  Front       POS
HOUBR_Front   HOUBR  8.700638e-08   600.0000            2  Front       POS
    Freight     WDF  0.000000e+00   145.0000            3  Front      ZERO

Total rows: 10

POSITION VECTORS BY PRODUCT (w_total for each product)

CLBR Position Vector (Total Qty: 2368.00):
  Node positions:
    A02: -268.00
 

In [107]:
# Build multi-product returns matrix
# Collect returns for all products, align dates, filter holidays

print("="*80)
print("BUILDING MULTI-PRODUCT RETURNS MATRIX")
print("="*80)

# Get products that have data
products_with_data = []
product_returns_dict = {}

for mapped_product in all_products:
    product_lower = mapped_to_data_column.get(mapped_product, mapped_product.lower())
    
    # Check if product data exists
    product_cols = [col for col in df_raw.columns if col.startswith(f'{product_lower}_A') and '/' not in col]
    if not product_cols:
        continue
    
    # Extract price data for this product
    product_cols = sorted(product_cols, key=lambda x: int(x.split('_A')[1]))
    df_prices = df_raw[product_cols].copy()
    
    # Compute daily changes (returns)
    df_returns = df_prices.diff().dropna()
    df_returns = df_returns.dropna()
    
    # Filter out holiday dates
    is_holiday = [date.date() in holiday_dates for date in df_returns.index]
    df_returns = df_returns[~pd.Series(is_holiday, index=df_returns.index)]
    
    if len(df_returns) < ewma_init_obs:
        print(f"  WARNING: Insufficient data for {mapped_product} ({len(df_returns)} < {ewma_init_obs}), skipping...")
        continue
    
    products_with_data.append(mapped_product)
    product_returns_dict[mapped_product] = df_returns
    print(f"  {mapped_product}: {len(df_returns)} observations, {len(product_cols)} nodes")

print(f"\nProducts with data: {products_with_data}")
print(f"Total products: {len(products_with_data)}")

# Find common dates across all products
if len(products_with_data) > 0:
    common_dates = product_returns_dict[products_with_data[0]].index
    for product in products_with_data[1:]:
        common_dates = common_dates.intersection(product_returns_dict[product].index)
    
    print(f"\nCommon dates across all products: {len(common_dates)}")
    print(f"Date range: {common_dates.min()} to {common_dates.max()}")
    
    # Build combined returns DataFrame
    combined_returns_list = []
    column_order = []
    
    for product in products_with_data:
        product_lower = mapped_to_data_column.get(product, product.lower())
        product_returns = product_returns_dict[product].loc[common_dates]
        
        # Get columns in order A01-A15
        product_cols = sorted([col for col in product_returns.columns], 
                             key=lambda x: int(x.split('_A')[1]))
        
        for col in product_cols:
            combined_returns_list.append(product_returns[col])
            column_order.append(col)
    
    # Create combined DataFrame
    combined_returns_df = pd.DataFrame(dict(zip(column_order, combined_returns_list)))
    combined_returns_df.index = common_dates
    
    # Create product_indices mapping: product -> (start_idx, end_idx) in combined matrix
    product_indices = {}
    current_idx = 0
    for product in products_with_data:
        product_indices[product] = (current_idx, current_idx + 15)
        current_idx += 15
        print(f"  {product}: columns {product_indices[product][0]}-{product_indices[product][1]-1}")
    
    print(f"\nCombined returns matrix shape: {combined_returns_df.shape}")
    print(f"Columns: {len(column_order)} ({len(products_with_data)} products × 15 nodes)")
    print(f"Rows: {len(common_dates)} (common trading dates)")
    print(f"\nProduct indices mapping created: {len(product_indices)} products")
else:
    print("ERROR: No products with sufficient data!")
    combined_returns_df = pd.DataFrame()
    product_indices = {}

BUILDING MULTI-PRODUCT RETURNS MATRIX
  CLBR: 968 observations, 15 nodes
  HOUBR: 964 observations, 15 nodes
  HTT: 966 observations, 15 nodes
  WDF: 259 observations, 15 nodes

Products with data: ['CLBR', 'HOUBR', 'HTT', 'WDF']
Total products: 4

Common dates across all products: 253
Date range: 2024-12-10 00:00:00 to 2025-12-24 00:00:00
  CLBR: columns 0-14
  HOUBR: columns 15-29
  HTT: columns 30-44
  WDF: columns 45-59

Combined returns matrix shape: (253, 60)
Columns: 60 (4 products × 15 nodes)
Rows: 253 (common trading dates)

Product indices mapping created: 4 products


### 8b. Compute Multi-Product EWMA Covariance Matrix

In [108]:
def compute_multi_product_ewma_covariance(combined_returns_df, products_with_data, product_indices, 
                                           front, mid, back, lambda_front, lambda_mid, lambda_back, 
                                           init_obs=60):
    """
    Compute multi-product EWMA covariance matrix with cross-product correlations.
    Uses bucket-specific lambdas for different tenor buckets.
    
    Parameters:
    -----------
    combined_returns_df : DataFrame
        Combined returns matrix with all products (columns: product_node, e.g., 'htt_A01')
    products_with_data : list
        List of products to include
    product_indices : dict
        Mapping from product to (start_idx, end_idx) in combined matrix
    front, mid, back : list
        Node lists for each bucket
    lambda_front, lambda_mid, lambda_back : float
        EWMA decay parameters for each bucket
    init_obs : int
        Number of observations for initial covariance
    
    Returns:
    --------
    Sigma_multi : ndarray
        Full multi-product covariance matrix
    """
    # Convert to numpy array
    returns_array = combined_returns_df.values  # Shape: (n_dates, n_products * 15)
    n_obs, n_vars = returns_array.shape
    n_products = len(products_with_data)
    
    if n_obs < init_obs:
        raise ValueError(f"Need at least {init_obs} observations, got {n_obs}")
    
    print(f"Computing multi-product EWMA covariance...")
    print(f"  Returns array shape: {returns_array.shape}")
    print(f"  Products: {n_products}")
    print(f"  Total variables: {n_vars} ({n_products} products × 15 nodes)")
    
    # Initialize with sample covariance of first N observations
    init_returns = returns_array[:init_obs]
    # Remove any rows with NaN
    valid_rows = ~np.isnan(init_returns).any(axis=1)
    init_returns = init_returns[valid_rows]
    
    if len(init_returns) < 10:
        # Fallback: use identity matrix scaled by variance
        cov_current = np.eye(n_vars) * np.var(returns_array, axis=0).mean()
        print(f"  WARNING: Using identity matrix initialization (only {len(init_returns)} valid rows)")
    else:
        cov_current = np.cov(init_returns.T)
        print(f"  Initialized with sample covariance from {len(init_returns)} observations")
    
    # Create lambda vector: one lambda per variable based on which bucket its node belongs to
    # Map each variable to its node and determine bucket
    lambda_vec = np.zeros(n_vars)
    
    for product in products_with_data:
        i_start, i_end = product_indices[product]
        product_lower = mapped_to_data_column.get(product, product.lower())
        
        # For each node in this product (A01-A15)
        for node_idx in range(15):
            var_idx = i_start + node_idx
            node_code = all_nodes[node_idx]
            
            # Determine which bucket this node belongs to
            if node_code in front:
                lambda_vec[var_idx] = lambda_front
            elif node_code in mid:
                lambda_vec[var_idx] = lambda_mid
            elif node_code in back:
                lambda_vec[var_idx] = lambda_back
            else:
                lambda_vec[var_idx] = lambda_back  # Default to back
    
    print(f"  Lambda distribution: Front={np.sum(lambda_vec == lambda_front)}, "
          f"Mid={np.sum(lambda_vec == lambda_mid)}, Back={np.sum(lambda_vec == lambda_back)}")
    
    # EWMA recursion: Σ_t = λ * Σ_{t-1} + (1-λ) * r_t * r_t'
    # But we use element-wise lambdas for bucket-specific decay
    for t in range(init_obs, n_obs):
        r_t = returns_array[t:t+1, :]  # Shape: (1, n_vars)
        
        # Skip if any NaN
        if np.isnan(r_t).any():
            continue
        
        # EWMA update with element-wise lambdas
        # For each element (i,j): Σ_{t}[i,j] = λ_ij * Σ_{t-1}[i,j] + (1-λ_ij) * r_t[i] * r_t[j]
        # Use average lambda for off-diagonal: λ_ij = (λ_i + λ_j) / 2
        # For diagonal: use λ_i directly
        
        # Compute outer product
        outer_product = np.outer(r_t[0], r_t[0])  # Shape: (n_vars, n_vars)
        
        # Create lambda matrix: average of row and column lambdas for off-diagonal
        lambda_matrix = np.outer(lambda_vec, np.ones(n_vars))
        lambda_matrix = (lambda_matrix + lambda_matrix.T) / 2  # Average for off-diagonal
        
        # EWMA update
        cov_current = lambda_matrix * cov_current + (1 - lambda_matrix) * outer_product
    
    print(f"  Final covariance matrix shape: {cov_current.shape}")
    print(f"  Completed EWMA recursion for {n_obs - init_obs} observations")
    
    return cov_current

# Compute multi-product EWMA covariance matrix
if len(products_with_data) > 0 and len(combined_returns_df) > 0:
    print("\n" + "="*80)
    print("COMPUTING MULTI-PRODUCT EWMA COVARIANCE MATRIX")
    print("="*80)
    
    Sigma_multi = compute_multi_product_ewma_covariance(
        combined_returns_df, products_with_data, product_indices,
        front, mid, back, lambda_front, lambda_mid, lambda_back, ewma_init_obs
    )
    
    print(f"\nMulti-product covariance matrix computed successfully!")
    print(f"  Shape: {Sigma_multi.shape}")
    print(f"  Size: {Sigma_multi.size:,} elements")
    
    # Check for cross-product correlations (sample)
    if len(products_with_data) >= 2:
        print(f"\nSample cross-product correlations (A01 nodes):")
        for i, prod_i in enumerate(products_with_data):
            for j, prod_j in enumerate(products_with_data):
                if i < j:  # Only show upper triangle
                    i_start, i_end = product_indices[prod_i]
                    j_start, j_end = product_indices[prod_j]
                    
                    # Get A01 node indices (first node)
                    i_node = i_start
                    j_node = j_start
                    
                    # Extract covariance and convert to correlation
                    cov_ij = Sigma_multi[i_node, j_node]
                    var_i = Sigma_multi[i_node, i_node]
                    var_j = Sigma_multi[j_node, j_node]
                    
                    if var_i > 0 and var_j > 0:
                        corr_ij = cov_ij / np.sqrt(var_i * var_j)
                        print(f"  {prod_i} vs {prod_j}: {corr_ij:.4f}")
else:
    print("ERROR: Cannot compute covariance - missing returns data!")
    Sigma_multi = None


COMPUTING MULTI-PRODUCT EWMA COVARIANCE MATRIX
Computing multi-product EWMA covariance...
  Returns array shape: (253, 60)
  Products: 4
  Total variables: 60 (4 products × 15 nodes)
  Initialized with sample covariance from 60 observations
  Lambda distribution: Front=16, Mid=16, Back=28
  Final covariance matrix shape: (60, 60)
  Completed EWMA recursion for 193 observations

Multi-product covariance matrix computed successfully!
  Shape: (60, 60)
  Size: 3,600 elements

Sample cross-product correlations (A01 nodes):
  CLBR vs HOUBR: 0.9100
  CLBR vs HTT: 0.0407
  CLBR vs WDF: -0.6046
  HOUBR vs HTT: 0.4514
  HOUBR vs WDF: -0.7580
  HTT vs WDF: -0.5254


In [109]:
print("="*80)
print("POSITION VECTOR ANALYSIS")
print("="*80)

position_analysis = []

for mapped_product in ['CLBR', 'HOUBR', 'HTT', 'WDF']:
    # Build position vector
    w_total = np.zeros(n_total)
    if mapped_product in delta_summary_df.columns:
        for _, row in delta_summary_df.iterrows():
            tenor = row['Tenor']
            position = row[mapped_product]
            if abs(position) > 1e-10 and tenor in contract_to_node:
                node = contract_to_node[tenor]
                node_idx = all_nodes.index(node)
                w_total[node_idx] = float(position)
    
    total_qty = np.sum(np.abs(w_total))
    net_qty = np.sum(w_total)
    n_nonzero = np.sum(np.abs(w_total) > 1e-10)
    
    # Check which buckets have positions
    front_pos = w_total[[all_nodes.index(n) for n in front]]
    mid_pos = w_total[[all_nodes.index(n) for n in mid]]
    back_pos = w_total[[all_nodes.index(n) for n in back]]
    
    front_qty = np.sum(np.abs(front_pos))
    mid_qty = np.sum(np.abs(mid_pos))
    back_qty = np.sum(np.abs(back_pos))
    
    position_analysis.append({
        'Product': mapped_product,
        'Total_abs_qty': total_qty,
        'Net_qty': net_qty,
        'N_nonzero_nodes': n_nonzero,
        'Front_qty': front_qty,
        'Mid_qty': mid_qty,
        'Back_qty': back_qty,
        'Max_position': np.max(np.abs(w_total)),
        'Max_position_node': all_nodes[np.argmax(np.abs(w_total))],
        'Concentration': np.max(np.abs(w_total)) / total_qty if total_qty > 0 else 0
    })
    
    print(f"\n{mapped_product} Position Analysis:")
    print(f"  Total absolute quantity: {total_qty:.2f}")
    print(f"  Net quantity: {net_qty:.2f}")
    print(f"  Non-zero nodes: {n_nonzero}")
    print(f"  Front bucket qty: {front_qty:.2f}")
    print(f"  Mid bucket qty: {mid_qty:.2f}")
    print(f"  Back bucket qty: {back_qty:.2f}")
    print(f"  Max position: {np.max(np.abs(w_total)):.2f} (node: {all_nodes[np.argmax(np.abs(w_total))]})")
    print(f"  Concentration ratio: {np.max(np.abs(w_total)) / total_qty if total_qty > 0 else 0:.2%}")

pos_analysis_df = pd.DataFrame(position_analysis)
print("\n" + "="*80)
print("POSITION ANALYSIS COMPARISON")
print("="*80)
print(pos_analysis_df.to_string(index=False))

POSITION VECTOR ANALYSIS

CLBR Position Analysis:
  Total absolute quantity: 2368.00
  Net quantity: 132.00
  Non-zero nodes: 5
  Front bucket qty: 268.00
  Mid bucket qty: 2000.00
  Back bucket qty: 100.00
  Max position: 850.00 (node: A07)
  Concentration ratio: 35.90%

HOUBR Position Analysis:
  Total absolute quantity: 3925.00
  Net quantity: 475.00
  Non-zero nodes: 12
  Front bucket qty: 3200.00
  Mid bucket qty: 200.00
  Back bucket qty: 525.00
  Max position: 1300.00 (node: A03)
  Concentration ratio: 33.12%

HTT Position Analysis:
  Total absolute quantity: 10549.00
  Net quantity: -749.00
  Non-zero nodes: 15
  Front bucket qty: 4874.00
  Mid bucket qty: 4600.00
  Back bucket qty: 1075.00
  Max position: 1974.00 (node: A03)
  Concentration ratio: 18.71%

WDF Position Analysis:
  Total absolute quantity: 145.00
  Net quantity: 15.00
  Non-zero nodes: 3
  Front bucket qty: 145.00
  Mid bucket qty: 0.00
  Back bucket qty: 0.00
  Max position: 65.00 (node: A02)
  Concentration ra

### 8c. Build Combined Position Vectors

In [110]:
# Build combined position vectors for all products
# w_total_combined: combined position vector [product1_A01, ..., product1_A15, product2_A01, ..., product2_A15, ...]

print("="*80)
print("BUILDING COMBINED POSITION VECTORS")
print("="*80)

if len(products_with_data) > 0 and 'product_indices' in globals():
    n_products = len(products_with_data)
    n_combined = n_products * 15  # Each product has 15 nodes
    
    # Initialize combined total position vector
    w_total_combined = np.zeros(n_combined)
    
    print(f"\nBuilding combined position vector for {n_products} products...")
    print(f"  Combined vector size: {n_combined} ({n_products} products × 15 nodes)")
    
    # Build position vector for each product and place in combined vector
    for product in products_with_data:
        if product not in product_indices:
            print(f"  WARNING: {product} not in product_indices, skipping...")
            continue
        
        i_start, i_end = product_indices[product]
        
        # Build single-product position vector
        w_product = np.zeros(15)
        if product in delta_summary_df.columns:
            for _, row in delta_summary_df.iterrows():
                tenor = row['Tenor']
                position = row[product]
                if abs(position) > 1e-10 and tenor in contract_to_node:
                    node = contract_to_node[tenor]
                    node_idx = all_nodes.index(node)
                    w_product[node_idx] = float(position)
        
        # Place in combined vector
        w_total_combined[i_start:i_end] = w_product
        
        total_qty = np.sum(np.abs(w_product))
        print(f"  {product}: columns {i_start}-{i_end-1}, total qty: {total_qty:.2f}")
    
    total_combined_qty = np.sum(np.abs(w_total_combined))
    print(f"\nCombined total position vector built successfully!")
    print(f"  Total absolute quantity across all products: {total_combined_qty:.2f}")
    print(f"  Non-zero elements: {np.sum(np.abs(w_total_combined) > 1e-10)}")
    
    # Display summary by product
    print(f"\nPosition summary by product in combined vector:")
    for product in products_with_data:
        if product in product_indices:
            i_start, i_end = product_indices[product]
            w_product = w_total_combined[i_start:i_end]
            total_qty = np.sum(np.abs(w_product))
            net_qty = np.sum(w_product)
            n_nonzero = np.sum(np.abs(w_product) > 1e-10)
            print(f"  {product}: abs_qty={total_qty:.2f}, net_qty={net_qty:.2f}, non_zero_nodes={n_nonzero}")
else:
    print("ERROR: Cannot build combined position vectors - missing products_with_data or product_indices!")
    w_total_combined = None

BUILDING COMBINED POSITION VECTORS

Building combined position vector for 4 products...
  Combined vector size: 60 (4 products × 15 nodes)
  CLBR: columns 0-14, total qty: 2368.00
  HOUBR: columns 15-29, total qty: 3925.00
  HTT: columns 30-44, total qty: 10549.00
  WDF: columns 45-59, total qty: 145.00

Combined total position vector built successfully!
  Total absolute quantity across all products: 16987.00
  Non-zero elements: 35

Position summary by product in combined vector:
  CLBR: abs_qty=2368.00, net_qty=132.00, non_zero_nodes=5
  HOUBR: abs_qty=3925.00, net_qty=475.00, non_zero_nodes=12
  HTT: abs_qty=10549.00, net_qty=-749.00, non_zero_nodes=15
  WDF: abs_qty=145.00, net_qty=15.00, non_zero_nodes=3


### 8b. Compute Multi-Product EWMA Covariance Matrix

### 8d. Recalculate MC with Multi-Product Covariance

## 6. Aggregation by Strategy and Product

In [111]:
# Bucket-level comparison between independent and shared covariance
if len(position_mc_multi_product) > 0:
    print("\n" + "="*80)
    print("MC BY BUCKET COMPARISON - INDEPENDENT vs SHARED COVARIANCE")
    print("="*80)
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame(position_mc_multi_product)
    
    # Add bucket information from original results
    bucket_map = {}
    for result in position_mc_results:
        key = (result['Strategy'], result['Product'])
        bucket_map[key] = result.get('Bucket', 'Unknown')
    
    comparison_df['Bucket'] = comparison_df.apply(
        lambda row: bucket_map.get((row['Strategy'], row['Product']), 'Unknown'), 
        axis=1
    )
    
    # Aggregate by bucket for both methods
    mc_by_bucket_original = comparison_df.groupby('Bucket').agg({
        'MC_original': 'sum'
    }).reset_index()
    mc_by_bucket_original.columns = ['Bucket', 'MC_independent']
    
    mc_by_bucket_multi = comparison_df.groupby('Bucket').agg({
        'MC_multi_product': 'sum'
    }).reset_index()
    mc_by_bucket_multi.columns = ['Bucket', 'MC_shared']
    
    # Merge and compare
    mc_bucket_comparison = mc_by_bucket_original.merge(mc_by_bucket_multi, on='Bucket', how='outer')
    mc_bucket_comparison = mc_bucket_comparison.fillna(0.0)
    mc_bucket_comparison['Difference'] = mc_bucket_comparison['MC_shared'] - mc_bucket_comparison['MC_independent']
    mc_bucket_comparison['Pct_change'] = (mc_bucket_comparison['Difference'] / 
                                          mc_bucket_comparison['MC_independent'].abs() * 100).replace([np.inf, -np.inf], 0.0)
    mc_bucket_comparison = mc_bucket_comparison.sort_values('Bucket')
    
    print("\nMC by Bucket - Independent vs Shared Covariance:")
    print(mc_bucket_comparison.to_string(index=False))
    
    # Show bucket-level impact
    print(f"\nBucket-level impact of cross-product correlations:")
    for _, row in mc_bucket_comparison.iterrows():
        if abs(row['Difference']) > 1:
            print(f"  {row['Bucket']}: {row['MC_independent']:.2f} → {row['MC_shared']:.2f} "
                  f"({row['Difference']:+.2f}, {row['Pct_change']:+.2f}%)")
    
    # Compare with direct bucket MC calculation if available
    if 'mc_by_bucket_shared' in globals() and len(mc_by_bucket_shared) > 0:
        print(f"\n" + "="*80)
        print("DIRECT BUCKET MC (from position vectors) vs AGGREGATED (from strategies)")
        print("="*80)
        
        # Merge direct calculation with aggregated
        bucket_direct_vs_agg = mc_by_bucket_shared[['Bucket', 'MC_to_total']].copy()
        bucket_direct_vs_agg.columns = ['Bucket', 'MC_direct']
        bucket_direct_vs_agg = bucket_direct_vs_agg.merge(
            mc_bucket_comparison[['Bucket', 'MC_shared']], 
            on='Bucket', 
            how='outer'
        ).fillna(0.0)
        bucket_direct_vs_agg['Difference'] = bucket_direct_vs_agg['MC_direct'] - bucket_direct_vs_agg['MC_shared']
        
        print(bucket_direct_vs_agg.to_string(index=False))
        print(f"\nNote: Direct calculation uses bucket position vectors directly.")
        print(f"      Aggregated uses sum of strategy/product MCs within each bucket.")
else:
    print("ERROR: No multi-product MC results available for bucket comparison!")

NameError: name 'position_mc_multi_product' is not defined

### 8e. Analysis and Comparison

In [112]:
# Analysis and comparison
if len(position_mc_multi_product) > 0:
    print("\n" + "="*80)
    print("MULTI-PRODUCT COVARIANCE ANALYSIS")
    print("="*80)
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame(position_mc_multi_product)
    
    # Calculate summary statistics
    total_mc_original = comparison_df['MC_original'].sum()
    total_mc_multi = comparison_df['MC_multi_product'].sum()
    total_difference = total_mc_multi - total_mc_original
    total_pct_change = (total_difference / abs(total_mc_original) * 100) if abs(total_mc_original) > 1e-10 else 0.0
    
    print(f"\nSUMMARY STATISTICS:")
    print(f"  Total MC (independence assumption): {total_mc_original:.2f}")
    print(f"  Total MC (multi-product covariance): {total_mc_multi:.2f}")
    print(f"  Difference: {total_difference:.2f}")
    print(f"  % Change: {total_pct_change:.2f}%")
    
    # Sort by absolute difference
    comparison_df['abs_difference'] = comparison_df['Difference'].abs()
    comparison_df = comparison_df.sort_values('abs_difference', ascending=False)
    
    print(f"\n" + "="*80)
    print("MC COMPARISON BY STRATEGY/PRODUCT")
    print("="*80)
    print(comparison_df[['Strategy', 'Product', 'MC_original', 'MC_multi_product', 
                        'Difference', 'Pct_change']].to_string(index=False))
    
    # Identify most affected strategies
    print(f"\n" + "="*80)
    print("MOST AFFECTED STRATEGIES/PRODUCTS")
    print("="*80)
    significant_changes = comparison_df[comparison_df['abs_difference'] > 100]  # > $100 difference
    if len(significant_changes) > 0:
        print(f"Strategies with difference > $100:")
        print(significant_changes[['Strategy', 'Product', 'MC_original', 'MC_multi_product', 
                                  'Difference', 'Pct_change']].to_string(index=False))
    else:
        print("No strategies with difference > $100")
    
    # Show strategies with >10% change
    large_pct_changes = comparison_df[comparison_df['Pct_change'].abs() > 10]
    if len(large_pct_changes) > 0:
        print(f"\nStrategies with >10% change:")
        print(large_pct_changes[['Strategy', 'Product', 'MC_original', 'MC_multi_product', 
                                'Difference', 'Pct_change']].to_string(index=False))
    else:
        print("\nNo strategies with >10% change")
    
    # Covariance matrix insights
    print(f"\n" + "="*80)
    print("CROSS-PRODUCT CORRELATION INSIGHTS")
    print("="*80)
    
    if len(products_with_data) >= 2:
        print("\nSample cross-product correlations (A01 nodes):")
        for i, prod_i in enumerate(products_with_data):
            for j, prod_j in enumerate(products_with_data):
                if i < j:  # Only show upper triangle
                    i_start, i_end = product_indices[prod_i]
                    j_start, j_end = product_indices[prod_j]
                    
                    # Get A01 node indices (first node)
                    i_node = i_start
                    j_node = j_start
                    
                    # Extract covariance and convert to correlation
                    cov_ij = Sigma_multi[i_node, j_node]
                    var_i = Sigma_multi[i_node, i_node]
                    var_j = Sigma_multi[j_node, j_node]
                    
                    if var_i > 0 and var_j > 0:
                        corr_ij = cov_ij / np.sqrt(var_i * var_j)
                        print(f"  {prod_i} vs {prod_j}: {corr_ij:.4f}")
        
        # Show average cross-product correlation
        cross_corrs = []
        for i, prod_i in enumerate(products_with_data):
            for j, prod_j in enumerate(products_with_data):
                if i != j:
                    i_start, i_end = product_indices[prod_i]
                    j_start, j_end = product_indices[prod_j]
                    
                    # Get diagonal of cross-product block
                    cross_block = Sigma_multi[i_start:i_end, j_start:j_end]
                    var_i_block = np.diag(Sigma_multi[i_start:i_end, i_start:i_end])
                    var_j_block = np.diag(Sigma_multi[j_start:j_end, j_start:j_end])
                    
                    # Average correlation across nodes
                    corrs = []
                    for k in range(15):
                        if var_i_block[k] > 0 and var_j_block[k] > 0:
                            corr_k = cross_block[k, k] / np.sqrt(var_i_block[k] * var_j_block[k])
                            corrs.append(corr_k)
                    
                    if len(corrs) > 0:
                        cross_corrs.append(np.mean(corrs))
        
        if len(cross_corrs) > 0:
            print(f"\nAverage cross-product correlation: {np.mean(cross_corrs):.4f}")
            print(f"  Min: {np.min(cross_corrs):.4f}")
            print(f"  Max: {np.max(cross_corrs):.4f}")
    
    print(f"\n" + "="*80)
    print("INTERPRETATION")
    print("="*80)
    print("""
The multi-product covariance matrix accounts for cross-product correlations:
- Positive correlations: Products move together, increasing total portfolio risk
- Negative correlations: Products move in opposite directions, reducing total risk (diversification benefit)
- The difference between independence and multi-product MC shows the impact of these correlations
    """)
else:
    print("ERROR: No multi-product MC results to analyze!")

NameError: name 'position_mc_multi_product' is not defined

In [113]:
# Aggregate by Strategy (across all products)
mc_by_strategy = position_mc_df.groupby('Strategy').agg({
    'MC_to_total': 'sum',
    'Qty_total': 'sum',
    'Tenor_count': 'sum',
    'Product': lambda x: ', '.join(sorted(x.unique()))
}).reset_index()
mc_by_strategy.columns = ['Strategy', 'MC_to_total', 'Qty_total', 'Tenor_count', 'Products']
mc_by_strategy['MC_signed'] = mc_by_strategy['MC_to_total'].apply(
    lambda x: 'POS' if x > 0 else 'NEG' if x < 0 else 'ZERO'
)
mc_by_strategy = mc_by_strategy.sort_values('MC_to_total', key=abs, ascending=False).reset_index(drop=True)

print("="*80)
print("MC AGGREGATION BY STRATEGY (across all products)")
print("="*80)
print(mc_by_strategy.to_string(index=False))

# Aggregate by Product (across all strategies)
mc_by_product = position_mc_df.groupby('Product').agg({
    'MC_to_total': 'sum',
    'Qty_total': 'sum',
    'Tenor_count': 'sum',
    'Strategy': lambda x: len(x.unique())
}).reset_index()
mc_by_product.columns = ['Product', 'MC_to_total', 'Qty_total', 'Tenor_count', 'Strategy_count']
mc_by_product['MC_signed'] = mc_by_product['MC_to_total'].apply(
    lambda x: 'POS' if x > 0 else 'NEG' if x < 0 else 'ZERO'
)
mc_by_product = mc_by_product.sort_values('MC_to_total', key=abs, ascending=False).reset_index(drop=True)

print("\n" + "="*80)
print("MC AGGREGATION BY PRODUCT (across all strategies)")
print("="*80)
print(mc_by_product.to_string(index=False))

MC AGGREGATION BY STRATEGY (across all products)
   Strategy   MC_to_total  Qty_total  Tenor_count    Products MC_signed
  HTT_Rolls  1.765134e+05  8900.0000           30         HTT       POS
HOUBR_rolls  3.508698e+04  5000.0000            8 CLBR, HOUBR       POS
 HOUBR_Back  1.936570e+04   756.9996           16 CLBR, HOUBR       POS
    HTT_Mid  1.641687e+04   574.0000            1         HTT       POS
   HTT_Back  5.510005e+03   975.0000           18         HTT       POS
  HTT_Front -2.860082e+03   100.0000            1         HTT       NEG
HOUBR_Front  8.700638e-08   600.0000            2       HOUBR       POS
    Freight  0.000000e+00   145.0000            3         WDF      ZERO

MC AGGREGATION BY PRODUCT (across all strategies)
Product   MC_to_total  Qty_total  Tenor_count  Strategy_count MC_signed
    HTT 195580.158615 10549.0000           50               4       POS
   CLBR  32900.115037  2431.9996            7               2       POS
  HOUBR  21552.570271  3925.0000    

## 7. Cross-Product MC Summary

## 4d. Comprehensive Comparison Table

In [114]:
# Cross-product summary
# Note: This assumes independence between products (sums MCs)
# For full cross-product correlation, would need multi-product covariance matrix

total_mc_all_products = position_mc_df['MC_to_total'].sum()
total_qty_all_products = position_mc_df['Qty_total'].sum()

print("="*80)
print("CROSS-PRODUCT MC SUMMARY")
print("="*80)
print(f"Total MC across all products (sum of single-product MCs): {total_mc_all_products:.2f}")
print(f"Total quantity across all products: {total_qty_all_products:.2f}")
print(f"\nNote: This assumes independence between products.")
print(f"      For full cross-product correlation, a multi-product covariance matrix would be needed.")

# Strategy × Product pivot table
strategy_product_pivot = position_mc_df.pivot_table(
    index='Strategy',
    columns='Product',
    values='MC_to_total',
    aggfunc='sum',
    fill_value=0.0
)

print("\n" + "="*80)
print("MC BY STRATEGY × PRODUCT (Pivot Table)")
print("="*80)
print(strategy_product_pivot.to_string())

# Top contributors
print("\n" + "="*80)
print("TOP 10 CONTRIBUTORS BY ABSOLUTE MC")
print("="*80)
top_contributors = position_mc_df.head(10)[['Strategy', 'Product', 'MC_to_total', 'Qty_total', 'Bucket']]
print(top_contributors.to_string(index=False))

CROSS-PRODUCT MC SUMMARY
Total MC across all products (sum of single-product MCs): 250032.84
Total quantity across all products: 17051.00

Note: This assumes independence between products.
      For full cross-product correlation, a multi-product covariance matrix would be needed.

MC BY STRATEGY × PRODUCT (Pivot Table)
Product              CLBR         HOUBR            HTT  WDF
Strategy                                                   
Freight          0.000000  0.000000e+00       0.000000  0.0
HOUBR_Back   -2186.869610  2.155257e+04       0.000000  0.0
HOUBR_Front      0.000000  8.700638e-08       0.000000  0.0
HOUBR_rolls  35086.984647  3.012441e-07       0.000000  0.0
HTT_Back         0.000000  0.000000e+00    5510.005152  0.0
HTT_Front        0.000000  0.000000e+00   -2860.081791  0.0
HTT_Mid          0.000000  0.000000e+00   16416.869483  0.0
HTT_Rolls        0.000000  0.000000e+00  176513.365772  0.0

TOP 10 CONTRIBUTORS BY ABSOLUTE MC
   Strategy Product   MC_to_total  Qty_tot

## 8. Final Summary

In [115]:
# Analysis and comparison
if len(position_mc_multi_product) > 0:
    print("\n" + "="*80)
    print("MULTI-PRODUCT COVARIANCE ANALYSIS")
    print("="*80)
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame(position_mc_multi_product)
    
    # Calculate summary statistics
    total_mc_original = comparison_df['MC_original'].sum()
    total_mc_multi = comparison_df['MC_multi_product'].sum()
    total_difference = total_mc_multi - total_mc_original
    total_pct_change = (total_difference / abs(total_mc_original) * 100) if abs(total_mc_original) > 1e-10 else 0.0
    
    print(f"\nSUMMARY STATISTICS:")
    print(f"  Total MC (independence assumption): {total_mc_original:.2f}")
    print(f"  Total MC (multi-product covariance): {total_mc_multi:.2f}")
    print(f"  Difference: {total_difference:.2f}")
    print(f"  % Change: {total_pct_change:.2f}%")
    
    # Sort by absolute difference
    comparison_df['abs_difference'] = comparison_df['Difference'].abs()
    comparison_df = comparison_df.sort_values('abs_difference', ascending=False)
    
    print(f"\n" + "="*80)
    print("MC COMPARISON BY STRATEGY/PRODUCT")
    print("="*80)
    print(comparison_df[['Strategy', 'Product', 'MC_original', 'MC_multi_product', 
                        'Difference', 'Pct_change']].to_string(index=False))
    
    # Identify most affected strategies
    print(f"\n" + "="*80)
    print("MOST AFFECTED STRATEGIES/PRODUCTS")
    print("="*80)
    significant_changes = comparison_df[comparison_df['abs_difference'] > 100]  # > $100 difference
    if len(significant_changes) > 0:
        print(f"Strategies with difference > $100:")
        print(significant_changes[['Strategy', 'Product', 'MC_original', 'MC_multi_product', 
                                  'Difference', 'Pct_change']].to_string(index=False))
    else:
        print("No strategies with difference > $100")
    
    # Show strategies with >10% change
    large_pct_changes = comparison_df[comparison_df['Pct_change'].abs() > 10]
    if len(large_pct_changes) > 0:
        print(f"\nStrategies with >10% change:")
        print(large_pct_changes[['Strategy', 'Product', 'MC_original', 'MC_multi_product', 
                                'Difference', 'Pct_change']].to_string(index=False))
    else:
        print("\nNo strategies with >10% change")
    
    # Covariance matrix insights
    print(f"\n" + "="*80)
    print("CROSS-PRODUCT CORRELATION INSIGHTS")
    print("="*80)
    
    if len(products_with_data) >= 2:
        print("\nSample cross-product correlations (A01 nodes):")
        for i, prod_i in enumerate(products_with_data):
            for j, prod_j in enumerate(products_with_data):
                if i < j:  # Only show upper triangle
                    i_start, i_end = product_indices[prod_i]
                    j_start, j_end = product_indices[prod_j]
                    
                    # Get A01 node indices (first node)
                    i_node = i_start
                    j_node = j_start
                    
                    # Extract covariance and convert to correlation
                    cov_ij = Sigma_multi[i_node, j_node]
                    var_i = Sigma_multi[i_node, i_node]
                    var_j = Sigma_multi[j_node, j_node]
                    
                    if var_i > 0 and var_j > 0:
                        corr_ij = cov_ij / np.sqrt(var_i * var_j)
                        print(f"  {prod_i} vs {prod_j}: {corr_ij:.4f}")
        
        # Show average cross-product correlation
        cross_corrs = []
        for i, prod_i in enumerate(products_with_data):
            for j, prod_j in enumerate(products_with_data):
                if i != j:
                    i_start, i_end = product_indices[prod_i]
                    j_start, j_end = product_indices[prod_j]
                    
                    # Get diagonal of cross-product block
                    cross_block = Sigma_multi[i_start:i_end, j_start:j_end]
                    var_i_block = np.diag(Sigma_multi[i_start:i_end, i_start:i_end])
                    var_j_block = np.diag(Sigma_multi[j_start:j_end, j_start:j_end])
                    
                    # Average correlation across nodes
                    corrs = []
                    for k in range(15):
                        if var_i_block[k] > 0 and var_j_block[k] > 0:
                            corr_k = cross_block[k, k] / np.sqrt(var_i_block[k] * var_j_block[k])
                            corrs.append(corr_k)
                    
                    if len(corrs) > 0:
                        cross_corrs.append(np.mean(corrs))
        
        if len(cross_corrs) > 0:
            print(f"\nAverage cross-product correlation: {np.mean(cross_corrs):.4f}")
            print(f"  Min: {np.min(cross_corrs):.4f}")
            print(f"  Max: {np.max(cross_corrs):.4f}")
    
    print(f"\n" + "="*80)
    print("INTERPRETATION")
    print("="*80)
    print("""
The multi-product covariance matrix accounts for cross-product correlations:
- Positive correlations: Products move together, increasing total portfolio risk
- Negative correlations: Products move in opposite directions, reducing total risk (diversification benefit)
- The difference between independence and multi-product MC shows the impact of these correlations
    """)
else:
    print("ERROR: No multi-product MC results to analyze!")

NameError: name 'position_mc_multi_product' is not defined

## 5a. MC Results in pos_summary Format

In [92]:
# Create MC lookup DataFrame from position_mc_results
mc_lookup_df = pd.DataFrame(position_mc_results)[['Strategy', 'Product', 'MC_to_total']]
mc_lookup_df = mc_lookup_df.rename(columns={'Product': 'Mapped_Product'})

# Start with pos_summary and add Mapped_Product
pos_summary_with_mc = pos_summary_df.copy()
pos_summary_with_mc['Mapped_Product'] = pos_summary_with_mc['Product'].map(product_to_mapped)

# Merge with MC values
pos_summary_with_mc = pos_summary_with_mc.merge(
    mc_lookup_df,
    on=['Strategy', 'Mapped_Product'],
    how='left'
)

# Fill missing MC values with 0.0
pos_summary_with_mc['MC_to_total'] = pos_summary_with_mc['MC_to_total'].fillna(0.0)

# Select final columns (drop Mapped_Product from output)
pos_summary_with_mc_output = pos_summary_with_mc[['Qty', 'Tenor', 'Product', 'Strategy', 'MC_to_total']]

# Display the merged dataframe
print("="*80)
print("MC RESULTS IN pos_summary FORMAT")
print("="*80)
print(f"\nTotal rows: {len(pos_summary_with_mc_output)}")
print(f"Rows with MC > 0: {len(pos_summary_with_mc_output[pos_summary_with_mc_output['MC_to_total'] > 0])}")
print(f"Rows with MC = 0: {len(pos_summary_with_mc_output[pos_summary_with_mc_output['MC_to_total'] == 0])}")
print(f"\nFirst 20 rows:")
print(pos_summary_with_mc_output.head(20).to_string(index=False))

# Export to CSV
output_file = 'pos_summary_with_mc.csv'
pos_summary_with_mc_output.to_csv(output_file, index=False)
print(f"\n{'='*80}")
print(f"Exported to: {output_file}")
print(f"{'='*80}")

# Show summary by Strategy/Product
print("\nMC Summary by Strategy and Product:")
print("-"*80)
summary = pos_summary_with_mc_output.groupby(['Strategy', 'Product'])['MC_to_total'].first().reset_index()
summary = summary.sort_values('MC_to_total', key=abs, ascending=False)
print(summary.to_string(index=False))

MC RESULTS IN pos_summary FORMAT

Total rows: 20
Rows with MC > 0: 12
Rows with MC = 0: 6

First 20 rows:
       Qty       Tenor     Product    Strategy   MC_to_total
  100.0000          H6         HTT   HTT_Front -2.860082e+03
 -574.0000          K6         HTT     HTT_Mid  1.641687e+04
  -75.0000       Cal27         HTT    HTT_Back  5.510005e+03
  100.0000       H2-26         HTT    HTT_Back  5.510005e+03
  300.0000       H6/J6 HOUBR_Cross HOUBR_Front  8.700638e-08
  -25.0000       Cal27       HOUBR  HOUBR_Back  2.155257e+04
   31.9996          J6        CLBR  HOUBR_Back -2.186870e+03
  200.0000       Q4-26       HOUBR  HOUBR_Back  2.155257e+04
-1400.0000 Q2-26/Q3-26   HTT Rolls   HTT_Rolls  1.765134e+05
 -100.0000 Cal27/Cal28   HTT Rolls   HTT_Rolls  1.765134e+05
-1300.0000       K6/M6 HOUBR Boxes HOUBR_rolls  3.012441e-07
 -300.0000       J6/N6  CLBR Boxes HOUBR_rolls  3.508698e+04
  100.0000       Z6/Z7  CLBR Boxes HOUBR_rolls  3.508698e+04
  850.0000       U6/V6  CLBR Boxes HOUBR

In [94]:
print("="*80)
print("POSITION MC REPORT - FINAL SUMMARY")
print("="*80)

print("\n1. BY STRATEGY:")
print(mc_by_strategy.to_string(index=False))

print("\n\n2. BY PRODUCT:")
print(mc_by_product.to_string(index=False))

print("\n\n3. DETAILED POSITION MC (first 20 rows):")
print(position_mc_df.head(20).to_string(index=False))

print("\n\n4. STRATEGY × PRODUCT PIVOT:")
print(strategy_product_pivot.to_string())

print("\n\n" + "="*80)
print("Report generation complete.")
print("="*80)

POSITION MC REPORT - FINAL SUMMARY

1. BY STRATEGY:
   Strategy   MC_to_total  Qty_total  Tenor_count    Products MC_signed
  HTT_Rolls  1.765134e+05  8900.0000           30         HTT       POS
HOUBR_rolls  3.508698e+04  5000.0000            8 CLBR, HOUBR       POS
 HOUBR_Back  1.936570e+04   756.9996           16 CLBR, HOUBR       POS
    HTT_Mid  1.641687e+04   574.0000            1         HTT       POS
   HTT_Back  5.510005e+03   975.0000           18         HTT       POS
  HTT_Front -2.860082e+03   100.0000            1         HTT       NEG
HOUBR_Front  8.700638e-08   600.0000            2       HOUBR       POS
    Freight  0.000000e+00   145.0000            3         WDF      ZERO


2. BY PRODUCT:
Product   MC_to_total  Qty_total  Tenor_count  Strategy_count MC_signed
    HTT 195580.158615 10549.0000           50               4       POS
   CLBR  32900.115037  2431.9996            7               2       POS
  HOUBR  21552.570271  3925.0000           19               3      

### 8a. Build Multi-Product Returns Matrix

In [119]:
# Calculate MC by bucket using shared covariance matrix
# For each bucket (Front, Mid, Back), compute MC contribution to total portfolio risk

print("\n" + "="*80)
print("MC BY BUCKET USING SHARED COVARIANCE MATRIX")
print("="*80)

if Sigma_multi is not None and w_total_combined is not None and len(products_with_data) > 0:
    # Build bucket position vectors in combined format
    # Each bucket vector contains positions in that bucket across all products
    
    # Get node indices for each bucket in the combined vector
    n_products = len(products_with_data)
    
    # Build bucket position vectors
    w_front_combined = np.zeros(len(w_total_combined))
    w_mid_combined = np.zeros(len(w_total_combined))
    w_back_combined = np.zeros(len(w_total_combined))
    
    # For each product, extract positions in each bucket
    for product in products_with_data:
        if product not in product_indices:
            continue
        
        i_start, i_end = product_indices[product]
        w_product = w_total_combined[i_start:i_end]
        
        # Map nodes to buckets
        for node_idx in range(15):
            var_idx = i_start + node_idx
            node_code = all_nodes[node_idx]
            
            if node_code in front:
                w_front_combined[var_idx] = w_product[node_idx]
            elif node_code in mid:
                w_mid_combined[var_idx] = w_product[node_idx]
            elif node_code in back:
                w_back_combined[var_idx] = w_product[node_idx]
    
    # Calculate MC for each bucket
    # MC_bucket = 1000 * (w_bucket' Σ_multi w_total) / sqrt(w_total' Σ_multi w_total)
    
    # Compute denominator (total portfolio risk)
    total_var = w_total_combined.T @ Sigma_multi @ w_total_combined
    if total_var <= 0:
        denominator = 1e-10
    else:
        denominator = np.sqrt(total_var)
    
    # Calculate MC for each bucket
    mc_front = 1000.0 * (w_front_combined.T @ Sigma_multi @ w_total_combined) / denominator
    mc_mid = 1000.0 * (w_mid_combined.T @ Sigma_multi @ w_total_combined) / denominator
    mc_back = 1000.0 * (w_back_combined.T @ Sigma_multi @ w_total_combined) / denominator
    
    # Calculate quantities for each bucket
    qty_front = np.sum(np.abs(w_front_combined))
    qty_mid = np.sum(np.abs(w_mid_combined))
    qty_back = np.sum(np.abs(w_back_combined))
    
    # Create summary DataFrame
    mc_by_bucket_shared = pd.DataFrame({
        'Bucket': ['Front', 'Mid', 'Back'],
        'MC_to_total': [mc_front, mc_mid, mc_back],
        'Qty_total': [qty_front, qty_mid, qty_back],
        'MC_signed': ['POS' if x > 0 else 'NEG' if x < 0 else 'ZERO' 
                      for x in [mc_front, mc_mid, mc_back]]
    })
    
    print(f"\nMC by Bucket (using shared covariance matrix):")
    print(mc_by_bucket_shared.to_string(index=False))
    
    print(f"\nTotal portfolio risk (denominator): {denominator:.2f}")
    print(f"Sum of bucket MCs: {mc_front + mc_mid + mc_back:.2f}")
    
    # Also aggregate strategies by bucket
    print(f"\n" + "="*80)
    print("MC BY BUCKET - AGGREGATED FROM STRATEGIES")
    print("="*80)
    
    if len(position_mc_multi_product) > 0:
        # Create DataFrame from multi-product MC results
        mc_multi_df = pd.DataFrame(position_mc_multi_product)
        
        # Add bucket information from original results
        bucket_map = {}
        for result in position_mc_results:
            key = (result['Strategy'], result['Product'])
            bucket_map[key] = result.get('Bucket', 'Unknown')
        
        mc_multi_df['Bucket'] = mc_multi_df.apply(
            lambda row: bucket_map.get((row['Strategy'], row['Product']), 'Unknown'), 
            axis=1
        )
        
        # Aggregate by bucket
        mc_by_bucket_strategies = mc_multi_df.groupby('Bucket').agg({
            'MC_multi_product': 'sum',
            'Qty_total': 'sum'
        }).reset_index()
        mc_by_bucket_strategies.columns = ['Bucket', 'MC_to_total', 'Qty_total']
        mc_by_bucket_strategies['MC_signed'] = mc_by_bucket_strategies['MC_to_total'].apply(
            lambda x: 'POS' if x > 0 else 'NEG' if x < 0 else 'ZERO'
        )
        mc_by_bucket_strategies = mc_by_bucket_strategies.sort_values('MC_to_total', key=abs, ascending=False)
        
        print("\nMC by Bucket (aggregated from strategy/product combinations):")
        print(mc_by_bucket_strategies.to_string(index=False))
        
        # Show breakdown by bucket and product
        print(f"\n" + "="*80)
        print("MC BY BUCKET × PRODUCT (using shared covariance)")
        print("="*80)
        
        # Add product info and aggregate
        mc_multi_df_with_bucket = mc_multi_df.copy()
        mc_bucket_product = mc_multi_df_with_bucket.groupby(['Bucket', 'Product']).agg({
            'MC_multi_product': 'sum',
            'Qty_total': 'sum'
        }).reset_index()
        mc_bucket_product.columns = ['Bucket', 'Product', 'MC_to_total', 'Qty_total']
        mc_bucket_product = mc_bucket_product.sort_values(['Bucket', 'MC_to_total'], key=lambda x: x.abs() if hasattr(x, 'abs') else [abs(y) for y in x], ascending=[True, False])
        
        print(mc_bucket_product.to_string(index=False))
    else:
        print("WARNING: No multi-product MC results available for strategy aggregation")
    
else:
    print("ERROR: Cannot calculate MC by bucket - missing covariance matrix, position vectors, or products!")
    mc_by_bucket_shared = pd.DataFrame()


MC BY BUCKET USING SHARED COVARIANCE MATRIX

MC by Bucket (using shared covariance matrix):
Bucket   MC_to_total  Qty_total MC_signed
 Front 130334.457618  8487.0004       POS
   Mid  13469.941541  6800.0000       POS
  Back  10703.128167  1700.0000       POS

Total portfolio risk (denominator): 154.51
Sum of bucket MCs: 154507.53

MC BY BUCKET - AGGREGATED FROM STRATEGIES

MC by Bucket (aggregated from strategy/product combinations):
Bucket   MC_to_total  Qty_total MC_signed
 Mixed 111622.793914 13000.0000       POS
 Front  42884.733412  4050.9996       POS

MC BY BUCKET × PRODUCT (using shared covariance)


TypeError: bad operand type for abs(): 'str'

In [117]:
# Recalculate MC for each strategy/product using multi-product covariance
if Sigma_multi is not None and w_total_combined is not None:
    print("\n" + "="*80)
    print("RECALCULATING MC WITH MULTI-PRODUCT COVARIANCE")
    print("="*80)
    
    position_mc_multi_product = []
    
    # Process each strategy/product combination
    for result in position_mc_results:
        strategy_name = result['Strategy']
        mapped_product = result['Product']
        
        # Skip if product not in products_with_data
        if mapped_product not in products_with_data:
            continue
        
        # Build combined strategy position vector
        w_strategy_combined = np.zeros(len(w_total_combined))
        
        # Get product index
        if mapped_product in product_indices:
            i_start, i_end = product_indices[mapped_product]
            
            # Build strategy position vector for this product
            w_strategy = build_strategy_position_vector(
                strategy_name, mapped_product, delta_positions_df,
                contract_to_node, all_nodes
            )
            
            # Place in appropriate position in combined vector
            w_strategy_combined[i_start:i_end] = w_strategy
            
            strategy_qty = np.sum(np.abs(w_strategy))
            if strategy_qty < 1e-10:
                continue  # Skip empty strategies
            
            # Calculate MC using multi-product covariance
            # MC = 1000 * (w_strategy' Σ_multi w_total) / sqrt(w_total' Σ_multi w_total)
            numerator = w_strategy_combined.T @ Sigma_multi @ w_total_combined
            denominator = np.sqrt(w_total_combined.T @ Sigma_multi @ w_total_combined)
            
            if denominator > 1e-10:
                mc_multi = 1000.0 * numerator / denominator
            else:
                mc_multi = 0.0
            
            # Get original MC for comparison
            mc_original = result['MC_to_total']
            
            position_mc_multi_product.append({
                'Strategy': strategy_name,
                'Product': mapped_product,
                'MC_original': mc_original,
                'MC_multi_product': mc_multi,
                'Difference': mc_multi - mc_original,
                'Pct_change': ((mc_multi - mc_original) / abs(mc_original) * 100) if abs(mc_original) > 1e-10 else 0.0,
                'Qty_total': strategy_qty
            })
    
    print(f"Recalculated MC for {len(position_mc_multi_product)} strategy/product combinations")
else:
    print("ERROR: Cannot recalculate MC - missing covariance matrix or position vectors!")
    position_mc_multi_product = []


RECALCULATING MC WITH MULTI-PRODUCT COVARIANCE
Recalculated MC for 10 strategy/product combinations


### 8a. Build Multi-Product Returns Matrix

### 8b. Compute Multi-Product EWMA Covariance Matrix

In [118]:
# Analysis and comparison
if len(position_mc_multi_product) > 0:
    print("\n" + "="*80)
    print("MULTI-PRODUCT COVARIANCE ANALYSIS")
    print("="*80)
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame(position_mc_multi_product)
    
    # Calculate summary statistics
    total_mc_original = comparison_df['MC_original'].sum()
    total_mc_multi = comparison_df['MC_multi_product'].sum()
    total_difference = total_mc_multi - total_mc_original
    total_pct_change = (total_difference / abs(total_mc_original) * 100) if abs(total_mc_original) > 1e-10 else 0.0
    
    print(f"\nSUMMARY STATISTICS:")
    print(f"  Total MC (independence assumption): {total_mc_original:.2f}")
    print(f"  Total MC (multi-product covariance): {total_mc_multi:.2f}")
    print(f"  Difference: {total_difference:.2f}")
    print(f"  % Change: {total_pct_change:.2f}%")
    
    # Sort by absolute difference
    comparison_df['abs_difference'] = comparison_df['Difference'].abs()
    comparison_df = comparison_df.sort_values('abs_difference', ascending=False)
    
    print(f"\n" + "="*80)
    print("MC COMPARISON BY STRATEGY/PRODUCT")
    print("="*80)
    print(comparison_df[['Strategy', 'Product', 'MC_original', 'MC_multi_product', 
                        'Difference', 'Pct_change']].to_string(index=False))
    
    # Identify most affected strategies
    print(f"\n" + "="*80)
    print("MOST AFFECTED STRATEGIES/PRODUCTS")
    print("="*80)
    significant_changes = comparison_df[comparison_df['abs_difference'] > 100]  # > $100 difference
    if len(significant_changes) > 0:
        print(f"Strategies with difference > $100:")
        print(significant_changes[['Strategy', 'Product', 'MC_original', 'MC_multi_product', 
                                  'Difference', 'Pct_change']].to_string(index=False))
    else:
        print("No strategies with difference > $100")
    
    # Show strategies with >10% change
    large_pct_changes = comparison_df[comparison_df['Pct_change'].abs() > 10]
    if len(large_pct_changes) > 0:
        print(f"\nStrategies with >10% change:")
        print(large_pct_changes[['Strategy', 'Product', 'MC_original', 'MC_multi_product', 
                                'Difference', 'Pct_change']].to_string(index=False))
    else:
        print("\nNo strategies with >10% change")
    
    # Covariance matrix insights
    print(f"\n" + "="*80)
    print("CROSS-PRODUCT CORRELATION INSIGHTS")
    print("="*80)
    
    if len(products_with_data) >= 2:
        print("\nSample cross-product correlations (A01 nodes):")
        for i, prod_i in enumerate(products_with_data):
            for j, prod_j in enumerate(products_with_data):
                if i < j:  # Only show upper triangle
                    i_start, i_end = product_indices[prod_i]
                    j_start, j_end = product_indices[prod_j]
                    
                    # Get A01 node indices (first node)
                    i_node = i_start
                    j_node = j_start
                    
                    # Extract covariance and convert to correlation
                    cov_ij = Sigma_multi[i_node, j_node]
                    var_i = Sigma_multi[i_node, i_node]
                    var_j = Sigma_multi[j_node, j_node]
                    
                    if var_i > 0 and var_j > 0:
                        corr_ij = cov_ij / np.sqrt(var_i * var_j)
                        print(f"  {prod_i} vs {prod_j}: {corr_ij:.4f}")
        
        # Show average cross-product correlation
        cross_corrs = []
        for i, prod_i in enumerate(products_with_data):
            for j, prod_j in enumerate(products_with_data):
                if i != j:
                    i_start, i_end = product_indices[prod_i]
                    j_start, j_end = product_indices[prod_j]
                    
                    # Get diagonal of cross-product block
                    cross_block = Sigma_multi[i_start:i_end, j_start:j_end]
                    var_i_block = np.diag(Sigma_multi[i_start:i_end, i_start:i_end])
                    var_j_block = np.diag(Sigma_multi[j_start:j_end, j_start:j_end])
                    
                    # Average correlation across nodes
                    corrs = []
                    for k in range(15):
                        if var_i_block[k] > 0 and var_j_block[k] > 0:
                            corr_k = cross_block[k, k] / np.sqrt(var_i_block[k] * var_j_block[k])
                            corrs.append(corr_k)
                    
                    if len(corrs) > 0:
                        cross_corrs.append(np.mean(corrs))
        
        if len(cross_corrs) > 0:
            print(f"\nAverage cross-product correlation: {np.mean(cross_corrs):.4f}")
            print(f"  Min: {np.min(cross_corrs):.4f}")
            print(f"  Max: {np.max(cross_corrs):.4f}")
    
    print(f"\n" + "="*80)
    print("INTERPRETATION")
    print("="*80)
    print("""
The multi-product covariance matrix accounts for cross-product correlations:
- Positive correlations: Products move together, increasing total portfolio risk
- Negative correlations: Products move in opposite directions, reducing total risk (diversification benefit)
- The difference between independence and multi-product MC shows the impact of these correlations
    """)
else:
    print("ERROR: No multi-product MC results to analyze!")


MULTI-PRODUCT COVARIANCE ANALYSIS

SUMMARY STATISTICS:
  Total MC (independence assumption): 250032.84
  Total MC (multi-product covariance): 154507.53
  Difference: -95525.32
  % Change: -38.21%

MC COMPARISON BY STRATEGY/PRODUCT
   Strategy Product   MC_original  MC_multi_product    Difference    Pct_change
  HTT_Rolls     HTT  1.765134e+05      95506.005492 -81007.360280 -4.589305e+01
HOUBR_rolls    CLBR  3.508698e+04       1786.180724 -33300.803923 -9.490928e+01
HOUBR_rolls   HOUBR  3.012441e-07      24986.764738  24986.764738  8.294525e+12
 HOUBR_Back   HOUBR  2.155257e+04      12950.160009  -8602.410262 -3.991362e+01
   HTT_Back     HTT  5.510005e+03       1380.447689  -4129.557463 -7.494653e+01
HOUBR_Front   HOUBR  8.700638e-08       3398.469168   3398.469168  3.906000e+12
 HOUBR_Back    CLBR -2.186870e+03        619.695714   2806.565324  1.283371e+02
    HTT_Mid     HTT  1.641687e+04      15314.743853  -1102.125630 -6.713373e+00
  HTT_Front     HTT -2.860082e+03      -2046.601